In [56]:
import os
import zipfile
import shutil
import numpy as np
import pandas as pd
from google.cloud import storage
from pathlib import Path
from tqdm import tqdm

storage_client = storage.Client()
bucket = storage_client.bucket('leo_font')

In [69]:
fds = {
    "zipfile": "/home/jupyter/ai_font/data/zipfiles",
    "seen128": "/home/jupyter/ai_font/data/raw/seens/png128/data/raw_png_128",
    "seen96": "/home/jupyter/ai_font/data/raw/seens/png96/data/raw_png_96",
    "unseen128": "/home/jupyter/ai_font/data/raw/unseens/png128",
    "unseen96": "/home/jupyter/ai_font/data/raw/unseens/png96",
}
files = {
    'all_korean': "/home/jupyter/ai_font/data/raw/all_korean.parquet",
    "zipfile128": "/home/jupyter/ai_font/data/zipfiles/raw_png_128.zip",
    "zipfile96": "/home/jupyter/ai_font/data/zipfiles/raw_png_96.zip",
}

for k, fd in fds.items():
    if not os.path.exists(fd):
        os.makedirs(fd)
        print(f"Folder Created: {fd}")

In [70]:
ak = pd.read_parquet(files['all_korean'])

In [71]:
easy = "가너디로마버소우즈치카토퍼후"
mid = "갹넌됻래몌벼슈양쟈챼켴텉픞핳"
hard = "겱냙뎳랛몊볍숎융쟧츣캷툛펆햙"

In [72]:
seens = [l for i,l in enumerate(easy) if i % 2 == 1]
seens += [l for i,l in enumerate(mid) if i % 2 == 0]
seens += [l for i,l in enumerate(hard) if i % 2 == 1]

unseens = [l for i,l in enumerate(easy) if i % 2 == 0]
unseens += [l for i,l in enumerate(mid) if i % 2 == 1]
unseens += [l for i,l in enumerate(hard) if i % 2 == 0]

test_fonts = ['twice dahyun_4','twice nayeon_6','UhBee Sunhong','UhBee Howl','SeoulHangang Jang B', "MapoDPP"]

In [24]:
# download zip files
zipfd = "/home/jupyter/ai_font/data/zipfiles/"
blob = bucket.blob("zipfiles/raw_png_128.zip")
blob.download_to_filename(zipfd+"raw_png_128.zip")
blob = bucket.blob("zipfiles/raw_png_96.zip")
blob.download_to_filename(zipfd+"raw_png_96.zip")

In [28]:
zipfile.ZipFile(zipfd+"raw_png_128.zip").extractall(fds['seen128'])
zipfile.ZipFile(zipfd+"raw_png_96.zip").extractall(fds['seen96'])

In [73]:
# Move unseens
for font in test_fonts:
    for unseen_idx in ak.loc[ak[0].isin(unseens)].index:
        try:
            shutil.move(fds['seen128']+f"/{font}__{unseen_idx}.png",fds['unseen128']+f"/{font}__{unseen_idx}.png")
            shutil.move(fds['seen96']+f"/{font}__{unseen_idx}.png",fds['unseen96']+f"/{font}__{unseen_idx}.png")
        except:
            pass